# Project No. 3 - Bouquet of Flowers
### Authors:
M. Alejandro Villalobos C.
Óscar Ruiz Ramirez
Sofía Vargas Aceves
### Fecha:
24 de Marzo, 2022
### Description:
Tercer proyecto Machine Learning. Bouquet of Flowers Classification
<br>Video disponible en: https://drive.google.com/file/d/16a9lxTxlBlv13Bh5LxRoAMB8-DwKilv2/view?usp=sharing
<br>Dataset, csv y .ows en: https://drive.google.com/drive/folders/1oqmh6m7Xvt7Y1qrx2FHEz0nlC74QCSJE?usp=sharing

## 1. DATA READ


In [ ]:
import pandas as pd

Dataset = pd.read_csv("./Embedded_images.csv")

Dataset.keys()

In [ ]:
Dataset = Dataset.drop(['image','image name', 'size', 'width','height'],axis=1)
Dataset['category'] = Dataset['category'].replace(['Camelia', 'Dahlia', 'Hydrangea', 'Lilies', 'Orchids', 'Peony', 'Ranunculus', 'Roses', 'Sunflowers', 'Tulips'], [0,1,2,3,4,5,6,7,8,9])
DataFrameFlowers = pd.DataFrame(Dataset)

## 2. Data Preprocessing

In [ ]:
X = DataFrameFlowers.drop(['category'], axis = 1)
Y = DataFrameFlowers['category']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=0)

## Model #1 Logistic Regression

### 3. Model Creation

In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=50.0 / y_train.shape[0], penalty="l1", solver="saga", tol=0.1)
LR_better = LogisticRegression(C=50.0 / y_train.shape[0], penalty="l2", solver="lbfgs", tol=1e-4)

### 4. Training: Adjust Model with Historic Data

In [ ]:
LR.fit(X_train, y_train)
LR_better.fit(X_train, y_train)

### 5. Prediction for New Data

In [ ]:
y_predictLR = LR.predict(X_test)
y_predictLR_better = LR_better.predict(X_test)

## Model #2 Convolutional Neural Network (CNN)

In [ ]:
#Aqui haremos las variables especialmente para el modelo CNN, ya que se necesita estar en 3 dimensiones
#Para training 201 x 2048 x 1, para test 51 x 2048 x 1
X_trainCNN = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_testCNN = X_test.values.reshape(X_test.shape[0], X_test.shape[1] ,1)

In [ ]:
X_train.shape

### 3. Model Creation

In [ ]:
import numpy as np
import keras 
from keras.models import Sequential
from keras.layers import  Dense, Conv1D, Flatten, MaxPooling1D, Dropout

#Para coincidir tamaños, se convierten las etiquetas en "one hot vectors"
y_trainCNN = keras.utils.to_categorical(np.asarray(y_train.factorize()[0]))
y_testCNN = keras.utils.to_categorical(np.asarray(y_test.factorize()[0]))

CNN = Sequential()
CNN.add(Conv1D(128, 3, activation='relu', input_shape=(2048,1)))
CNN.add(MaxPooling1D((2)))
CNN.add(Conv1D(256, 3, activation='relu'))
CNN.add(MaxPooling1D((2)))
CNN.add(Conv1D(256, 3, activation='relu'))
CNN.add(MaxPooling1D((2)))

CNN.add(Flatten())
CNN.add(Dense(256, activation='relu'))

#Capa Dense tamaño 10 por las 10 categorizaciones finales
CNN.add(Dense(10, activation='softmax'))

CNN.compile(optimizer='adam', loss = keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
CNN_better = Sequential()
CNN_better.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(2048,1)))
CNN_better.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
CNN_better.add(Dropout(0.7))
CNN_better.add(MaxPooling1D(pool_size=2))
CNN_better.add(Flatten())
CNN_better.add(Dense(256, activation='relu'))

#Capa Dense tamaño 10 por las 10 categorizaciones finales
CNN_better.add(Dense(10, activation='softmax'))

CNN_better.compile(optimizer='adam', loss = keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
y_trainCNN.shape

### 4. Training: Adjust Model with Historic Data

In [ ]:
CNN_H = CNN.fit(X_trainCNN, y_trainCNN, validation_data=(X_testCNN, y_testCNN), epochs=10)

In [ ]:
CNN_HBetter = CNN_better.fit(X_trainCNN, y_trainCNN, validation_data=(X_testCNN, y_testCNN), epochs=10)

### 5. Prediction for New Data

In [ ]:
y_predictCNN = CNN.predict_classes(X_testCNN)
y_predictCNN_better = CNN_better.predict_classes(X_testCNN)

## Model #3 Support Vector Machine (SVM)

### 3. Model Creation

In [ ]:
from sklearn import svm
SVM = svm.SVC(kernel='poly')
SVM_better = svm.SVC(kernel='linear')

### 4. Training: Adjust Model with Historic Data

In [ ]:
SVM.fit(X_train, y_train)
SVM_better.fit(X_train, y_train)

### 5. Prediction for New Data

In [ ]:
y_predictSVM = SVM.predict(X_test)
y_predictSVM_better = SVM_better.predict(X_test)

## Model #4 Random Forest

### 3. Model Creation

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=500)
RFC_better = RandomForestClassifier(n_estimators= 400,bootstrap= False)

### 4. Training: Adjust Model with Historic Data

In [ ]:
RFC.fit(X_train, y_train)
RFC_better.fit(X_train, y_train)

### 5. Prediction for New Data

In [ ]:
y_predictRF = RFC.predict(X_test)
y_predictRF_better = RFC_better.predict(X_test)

## 6. Visualization of Results

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

### Model #1 Logistic Regression

In [ ]:
CV_LR = cross_val_score(LR, X_train, y_train, cv=3, scoring = "accuracy")
CV_LR_better = cross_val_score(LR_better, X_train, y_train, cv=3, scoring = "accuracy")

In [ ]:
AS_LR = accuracy_score(y_test,y_predictLR)
AS_LR_better = accuracy_score(y_test,y_predictLR_better)

### Model #2 Support Vector Machine (SVM)

In [ ]:
CV_SVM = cross_val_score(SVM, X_train, y_train, cv=3, scoring = "accuracy")
CV_SVM_better = cross_val_score(SVM_better, X_train, y_train, cv=3, scoring = "accuracy")

In [ ]:
AS_SVM = accuracy_score(y_test,y_predictSVM)
AS_SVM_better = accuracy_score(y_test,y_predictSVM_better)

### Model #3 Random Forest

In [ ]:
CV_RF = cross_val_score(RFC, X_train, y_train, cv=3, scoring = "accuracy")
CV_RF_better = cross_val_score(RFC_better, X_train, y_train, cv=3, scoring = "accuracy")

In [ ]:
AS_RF = accuracy_score(y_test,y_predictRF)
AS_RF_better = accuracy_score(y_test,y_predictRF_better)

### Model #5 Convolutional Neural Network (CNN)

In [ ]:
CV_CNN = CNN.evaluate(X_trainCNN, y_trainCNN)
CV_CNN_better = CNN_better.evaluate(X_trainCNN, y_trainCNN)

In [ ]:
AS_CNN = CNN.evaluate(X_testCNN, y_testCNN)
AS_CNN_better = CNN_better.evaluate(X_testCNN, y_testCNN)

In [ ]:
#Creación de dataframe con los resultados
models = ["LR","SVM", "RF", "CNN","LR_better", "SVM_better", "RF_better", "CNN_better"]
AS_values = [AS_LR, AS_SVM, AS_RF,  AS_CNN[1], AS_LR_better, AS_SVM_better, AS_RF_better, AS_CNN_better[1]]
CV_values = [CV_LR, CV_SVM, CV_RF, [CV_CNN[1],'-','-'],CV_LR_better,CV_SVM_better, CV_RF_better, [CV_CNN_better[1],'-','-']]

data = {"Accuracy Scores": AS_values, "Cross Validations": CV_values}
df = pd.DataFrame(data, index = models)

#Separación de arrays de Cross Validations en columnas
dfCV = pd.DataFrame(df['Cross Validations'].to_list(), columns=['CV #1','CV #2','CV #3'], index = models)
#Cálculo de promedio de Cross Validations
dfCV.insert(0,'CV AVG', dfCV[['CV #1','CV #2','CV #3']].mean(axis=1, numeric_only=True))

#Concatenación de dataframes con las columnas finales
dfAcc = pd.concat([df['Accuracy Scores'], dfCV], axis=1)
dfAcc = dfAcc.style.set_caption("Models' Accuracy Scores and Cross Validations")
dfAcc

## Confusion Matrix


In [ ]:
from sklearn.metrics import confusion_matrix
cmLR=confusion_matrix(y_test,y_predictLR)
print(cmLR)

print("--------------------------------")
cmLR_better=confusion_matrix(y_test,y_predictLR_better)
print(cmLR_better)

Se puede apreciar en esta matriz de confusion que al sumar la cantidad de positivos, osea de los valores correctos que mostro (La diagonal) nos dio mayor cantidad el segundo modelo de logistic Regression. 

In [ ]:
cmCNN=confusion_matrix(y_test,y_predictCNN)
print(cmCNN)

print("--------------------------------")
cmCNN_better=confusion_matrix(y_test,y_predictCNN_better)
print(cmCNN_better)

En las matrices de confusión de CNN se puede observar que no hubo muchas flores clasificadas correctamente, pues el accuracy de los modelos fue también bajo. No hubo mucha variación entre los resultados de cada una, puesto que su diagonal fue similar.

In [ ]:
cmSVM=confusion_matrix(y_test,y_predictSVM)
print(cmSVM)

print("--------------------------------")
cmSVM_better=confusion_matrix(y_test,y_predictSVM_better)
print(cmSVM_better)

Se puede apreciar en esta matriz de confusion que al sumar la cantidad de positivos, osea de los valores correctos que mostro (La diagonal) nos dio mayor cantidad el segundo modelo de Support Vector Machine. 

In [ ]:
cmRF=confusion_matrix(y_test,y_predictRF)
print(cmRF)

print("--------------------------------")
cmRF_better=confusion_matrix(y_test,y_predictRF_better)
print(cmRF_better)

Se puede observar que Random Forest en ambos modelos clasificó correctamente la mayoría de los datos, aunque también se aprecia una mejora del primer al segundo modelo al tener menos valores fuera de la diagonal.

## Preguntas

### ¿Con qué tipo de flor se lograron los mejores resultados?

In [ ]:
def bestFlowerPerModel(cm):
    diagonal = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        diagonal[i] = cm[i][i]

    sumRow = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        for j in range(10):
            sumRow[i] += cm[i][j]

    avg = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        avg[i] = diagonal[i]/sumRow[i]

    score = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        score[i] = avg[i]*(sumRow[i]/51)

    highest = 0
    pos = 0
    for i in range(10):
        if score[i] > highest:
            highest = score[i]
            pos = i

    return "The best flower is: " + str(pos) + " with a score of " + str(highest)

print(bestFlowerPerModel(cmLR))
print(bestFlowerPerModel(cmLR_better))
print(bestFlowerPerModel(cmCNN))
print(bestFlowerPerModel(cmCNN_better))
print(bestFlowerPerModel(cmSVM))
print(bestFlowerPerModel(cmSVM_better))
print(bestFlowerPerModel(cmRF))
print(bestFlowerPerModel(cmRF_better))

Podemos observar que las flores mejor clasificadas fueron las Hydrangeas (2) y las Rosas (7)

### ¿Cuál flor fue la mas difícil de clasificar?

In [ ]:
def worstFlowerPerModel(cm):
    diagonal = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        diagonal[i] = cm[i][i]

    sumRow = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        for j in range(10):
            sumRow[i] += cm[i][j]

    avg = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        avg[i] = diagonal[i]/sumRow[i]

    score = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        score[i] = avg[i]*(sumRow[i]/51)

    lowest = score[0]
    pos = 0
    for i in range(10):
        if score[i] < lowest:
            highest = score[i]
            pos = i

    return "The worst flower is: " + str(pos) + " with a score of " + str(lowest)

print(worstFlowerPerModel(cmLR))
print(worstFlowerPerModel(cmLR_better))
print(worstFlowerPerModel(cmCNN))
print(worstFlowerPerModel(cmCNN_better))
print(worstFlowerPerModel(cmSVM))
print(worstFlowerPerModel(cmSVM_better))
print(worstFlowerPerModel(cmRF))
print(worstFlowerPerModel(cmRF_better))

Aquí vemos que la flor peor categorizada evidentemente fue el girasol (8)

### ¿Cuál modelo obtuvo los mejores resultados?

In [ ]:
#Creación de dataframe con los resultados
models = ["LR","SVM", "RF", "CNN","LR_better", "SVM_better", "RF_better", "CNN_better"]
AS_values = [AS_LR, AS_SVM, AS_RF,  AS_CNN[1], AS_LR_better, AS_SVM_better, AS_RF_better, AS_CNN_better[1]]
CV_values = [CV_LR, CV_SVM, CV_RF, [CV_CNN[1],'-','-'],CV_LR_better,CV_SVM_better, CV_RF_better, [CV_CNN_better[1],'-','-']]

data = {"Accuracy Scores": AS_values, "Cross Validations": CV_values}
df = pd.DataFrame(data, index = models)

#Separación de arrays de Cross Validations en columnas, referenciando (10)
dfCV = pd.DataFrame(df['Cross Validations'].to_list(), columns=['CV #1','CV #2','CV #3'], index = models)
#Cálculo de promedio de Cross Validations
dfCV.insert(0,'CV AVG', dfCV[['CV #1','CV #2','CV #3']].mean(axis=1, numeric_only=True))

#Concatenación de dataframes con las columnas finales, referenciando (11)
dfAcc = pd.concat([df['Accuracy Scores'], dfCV], axis=1)
dfAcc = dfAcc.style.set_caption("Models' Accuracy Scores and Cross Validations")
dfAcc

El modelo con mejores resultados fue el RF_better
En segundo tenemos a LR_better, con una diferencia muy pequeña

### ¿Cómo cree que pueda mejorar los resultados obtenidos?

La mejor manera de mejorar nuestros resultados sería incrementar el tamaño de nuestro dataset, ya que tenemos muy pocos datos
Otra manera de mejorar los resultados podría ser continuar ajustando los parámetros de los modelos hasta encontrar algunos que den resultados aun mejores, o incluso cambiar de modelos que se adapten más a los datos.

### ¿Cómo varían los resultados entre el set de datos de entrenamiento y el de pruebas?

En la mayoría de los modelos los resultados de entrenamiento y de pruebas son muy similares.
Sin embargo, en el modelo de CNN observamos resultados de entrenamiento muy altos, pero en pruebas resultados muy bajos
Esto nos indica que ocurrió overfitting, a pesar de que utilizamos dropout para intentar contrarrestar este efecto

## References

>_Se basó en apuntes de Regresión Lineal y teoría del curso para este proyecto._

_Para todos los modelos excepto CNN_:
<br>
1. Cournapeau, D. (2021). scikit-learn, Machine Learning in Python. Retrieved from: https://scikit-learn.org/stable/


 _Para CNN_:  
 
2. Chollet, F. (2022). Clasificacion Básica: Predecir una imagen de moda. Retrieved from: https://www.tensorflow.org/tutorials/keras/classification?hl=es-419#hacer_predicciones

3. TensorFlow. (2022). Red neuronal convolucional (CNN). Retrieved from: https://www.tensorflow.org/tutorials/images/cnn
4. Ruizendaal, R. (2017).Deep Learning #3: More on CNNs & Handling Overfitting. Retrieved from: https://towardsdatascience.com/deep-learning-3-more-on-cnns-handling-overfitting-2bd5d99abe5d 
5. Brownlee, J. (2020). 1D Convolutional Neural Network Models for Human Activity Recognition. Retrieved from: https://machinelearningmastery.com/cnn-models-for-human-activity-recognition-time-series-classification/

6. Pandas. (2022). DataFrame. Retrieved from: https://pandas.pydata.org/docs/reference/frame.html

>_Se basó en apuntes de Regresión Lineal y teoría del curso para este proyecto._

_Para todos los modelos excepto CNN_:
<br>
1. Cournapeau, D. (2021). scikit-learn, Machine Learning in Python. Retrieved from: https://scikit-learn.org/stable/


 _Para CNN_:  
 
2. Chollet, F. (2022). Clasificacion Básica: Predecir una imagen de moda. Retrieved from: https://www.tensorflow.org/tutorials/keras/classification?hl=es-419#hacer_predicciones

3. TensorFlow. (2022). Red neuronal convolucional (CNN). Retrieved from: https://www.tensorflow.org/tutorials/images/cnn
4. Ruizendaal, R. (2017).Deep Learning #3: More on CNNs & Handling Overfitting. Retrieved from: https://towardsdatascience.com/deep-learning-3-more-on-cnns-handling-overfitting-2bd5d99abe5d 
5. Brownlee, J. (2020). 1D Convolutional Neural Network Models for Human Activity Recognition. Retrieved from: https://machinelearningmastery.com/cnn-models-for-human-activity-recognition-time-series-classification/

6. Pandas. (2022). DataFrame. Retrieved from: https://pandas.pydata.org/docs/reference/frame.html